# Lab 19: Cloud Security AI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab19_cloud_security.ipynb)

Build AI-powered multi-cloud security tools.

## Learning Objectives
- AWS CloudTrail log analysis
- Azure and GCP security monitoring
- Multi-cloud threat detection
- Cloud-native security automation

In [ ]:
!pip install pandas anthropic -q

In [ ]:
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict

## 1. AWS CloudTrail Analysis

In [ ]:
# Sample CloudTrail events
CLOUDTRAIL_EVENTS = [
    {
        "eventTime": "2024-01-15T10:00:00Z",
        "eventName": "ConsoleLogin",
        "userIdentity": {"type": "IAMUser", "userName": "admin"},
        "sourceIPAddress": "203.0.113.50",
        "responseElements": {"ConsoleLogin": "Success"}
    },
    {
        "eventTime": "2024-01-15T10:05:00Z",
        "eventName": "CreateAccessKey",
        "userIdentity": {"type": "IAMUser", "userName": "admin"},
        "sourceIPAddress": "203.0.113.50",
        "requestParameters": {"userName": "backdoor-user"}
    },
    {
        "eventTime": "2024-01-15T10:10:00Z",
        "eventName": "AttachUserPolicy",
        "userIdentity": {"type": "IAMUser", "userName": "admin"},
        "requestParameters": {
            "userName": "backdoor-user",
            "policyArn": "arn:aws:iam::aws:policy/AdministratorAccess"
        }
    },
    {
        "eventTime": "2024-01-15T10:15:00Z",
        "eventName": "DescribeInstances",
        "userIdentity": {"type": "IAMUser", "userName": "backdoor-user"},
        "sourceIPAddress": "198.51.100.25"  # Different IP!
    },
]

df = pd.DataFrame(CLOUDTRAIL_EVENTS)
print("CloudTrail Events:")
df[['eventTime', 'eventName', 'sourceIPAddress']]

## 2. Cloud Threat Detection Rules

In [ ]:
# Cloud security detection rules
DETECTION_RULES = [
    {
        "name": "IAM Persistence",
        "events": ["CreateAccessKey", "CreateUser", "AttachUserPolicy"],
        "severity": "HIGH",
        "mitre": "T1136.003"
    },
    {
        "name": "Privilege Escalation",
        "events": ["AttachUserPolicy", "AttachRolePolicy", "PutUserPolicy"],
        "policy_patterns": ["AdministratorAccess", "PowerUserAccess"],
        "severity": "CRITICAL",
        "mitre": "T1078.004"
    },
    {
        "name": "Resource Enumeration",
        "events": ["DescribeInstances", "ListBuckets", "DescribeSecurityGroups"],
        "threshold": 10,  # More than 10 in 5 minutes
        "severity": "MEDIUM",
        "mitre": "T1580"
    },
]

def detect_cloud_threats(events: List[Dict], rules: List[Dict]) -> List[Dict]:
    """Apply detection rules to CloudTrail events."""
    alerts = []
    
    for event in events:
        for rule in rules:
            if event['eventName'] in rule['events']:
                # Check for policy patterns if specified
                if 'policy_patterns' in rule:
                    policy = str(event.get('requestParameters', {}))
                    if not any(p in policy for p in rule['policy_patterns']):
                        continue
                
                alerts.append({
                    "rule": rule['name'],
                    "event": event['eventName'],
                    "severity": rule['severity'],
                    "mitre": rule['mitre'],
                    "time": event['eventTime']
                })
    
    return alerts

# Detect threats
alerts = detect_cloud_threats(CLOUDTRAIL_EVENTS, DETECTION_RULES)
print("Detected Threats:")
for alert in alerts:
    print(f"  [{alert['severity']}] {alert['rule']}: {alert['event']} ({alert['mitre']})")

## 3. Multi-Cloud Normalization

In [ ]:
# Normalize events from different cloud providers

def normalize_aws_event(event: Dict) -> Dict:
    """Normalize AWS CloudTrail event."""
    return {
        "cloud": "AWS",
        "timestamp": event.get('eventTime'),
        "action": event.get('eventName'),
        "actor": event.get('userIdentity', {}).get('userName', 'unknown'),
        "source_ip": event.get('sourceIPAddress'),
        "resource": event.get('requestParameters', {}),
        "outcome": "success" if 'Success' in str(event.get('responseElements', '')) else "unknown"
    }

def normalize_azure_event(event: Dict) -> Dict:
    """Normalize Azure Activity Log event."""
    return {
        "cloud": "Azure",
        "timestamp": event.get('time'),
        "action": event.get('operationName'),
        "actor": event.get('caller'),
        "source_ip": event.get('callerIpAddress'),
        "resource": event.get('resourceId'),
        "outcome": event.get('status', {}).get('value', 'unknown')
    }

# Example normalized events
normalized = [normalize_aws_event(e) for e in CLOUDTRAIL_EVENTS]
print("Normalized Events:")
for n in normalized[:2]:
    print(json.dumps(n, indent=2))

## 4. LLM-Powered Cloud Security Analysis

In [ ]:
from anthropic import Anthropic

def analyze_cloud_activity(events: List[Dict]) -> str:
    """Use LLM to analyze cloud activity for threats."""
    
    client = Anthropic()
    
    prompt = f"""
    Analyze these AWS CloudTrail events for security threats.
    
    EVENTS:
    {json.dumps(events, indent=2)}
    
    Provide:
    1. Timeline of suspicious activity
    2. Likely attack scenario
    3. MITRE ATT&CK techniques used
    4. Recommended response actions
    5. Detection rules to prevent future attacks
    """
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=2000,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.content[0].text

# Uncomment to run with API key
# analysis = analyze_cloud_activity(CLOUDTRAIL_EVENTS)
# print(analysis)

## 5. Attack Pattern Detection

In [ ]:
# Common cloud attack patterns
ATTACK_PATTERNS = {
    "iam_persistence": {
        "sequence": ["CreateUser", "CreateAccessKey", "AttachUserPolicy"],
        "window_minutes": 30,
        "description": "Creation of backdoor IAM user with privileges"
    },
    "data_exfiltration": {
        "sequence": ["DescribeInstances", "ListBuckets", "GetObject"],
        "window_minutes": 60,
        "description": "Enumeration followed by data access"
    },
    "crypto_mining": {
        "sequence": ["RunInstances", "RunInstances", "RunInstances"],
        "window_minutes": 5,
        "description": "Rapid instance creation for mining"
    }
}

def detect_attack_pattern(events: List[Dict], patterns: Dict) -> List[Dict]:
    """Detect multi-step attack patterns."""
    detected = []
    event_names = [e['eventName'] for e in events]
    
    for pattern_name, pattern in patterns.items():
        sequence = pattern['sequence']
        
        # Check if sequence exists (simplified)
        matched = 0
        for seq_event in sequence:
            if seq_event in event_names:
                matched += 1
        
        if matched >= len(sequence) * 0.7:  # 70% match
            detected.append({
                "pattern": pattern_name,
                "description": pattern['description'],
                "confidence": matched / len(sequence)
            })
    
    return detected

patterns = detect_attack_pattern(CLOUDTRAIL_EVENTS, ATTACK_PATTERNS)
print("Detected Attack Patterns:")
for p in patterns:
    print(f"  {p['pattern']}: {p['description']} (confidence: {p['confidence']:.0%})")

## Key Takeaways

1. **CloudTrail Analysis**: Parse and analyze AWS audit logs
2. **Detection Rules**: Create rules for common cloud threats
3. **Multi-Cloud**: Normalize events across providers
4. **Attack Patterns**: Detect multi-step attacks

## Course Complete!

You've completed all 22 labs in the AI Security Training Program. 
Now apply these skills to build your own security tools!